###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

# Decennial 2020 Redistricting Data 

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [3]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE PL2020.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [4]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [5]:
dfs = [dg1, dg2]
droppers = ['ID', 'Category', 'Sub Category']
for df in dfs:
    df.drop(droppers, axis = 1, inplace = True)

In [146]:
#url string and list parameters for column head and tail
url_str= 'https://api.census.gov/data/2020/dec/pl?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
tail_cols3 = 'GeoFIPS2'

In [147]:
splits = ['60280', '63140', '48980', '80200', '73460', '29920', '70580']

In [148]:
results = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list = deque(col_list)
        col_list.appendleft(head2)
        col_list.appendleft(head1)
        predicates= {} #call for all counties in state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:47"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        #df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
        predicates= {} #call for all counties in state of KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:21"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
        df = pd.concat([df, kycos], axis = 0)
        predicates= {} #call for all places in TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places in the region
        df = pd.concat([df, places], axis = 0)
        predicates= {} #call for all places in KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:21"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places of concern in KY
        df = pd.concat([df, places], axis = 0)
        col_list.remove(tail_cols2) #adjust the column list for different formats of geos
        predicates= {} #call for state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, state], axis = 0)
        predicates= {} #call for US
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "us:*"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, national], axis = 0)
        results.append(df)
new_df = pd.concat(results, axis = 1)
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)
print('Okay Finished')

Okay Finished


In [149]:
new_df.head()

,NAME,GEO_ID,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaska

In [150]:
part_one = new_df

In [151]:
#this works
data_appended = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list.append(tail_cols3)
        col_list = deque(col_list)
        col_list.appendleft(head2)  
        col_list.appendleft(head1)
        temp = []
        for thingy in splits:
            predicates= {}
            get_vars= var_list
            predicates["get"]= ",". join(get_vars)
            predicates["for"]= 'county (or part):*'
            predicates["in"] = "state:47, place:{}".format(thingy)
            data = requests.get(url_str, params= predicates)
            col_names = col_list
            data = pd.DataFrame(columns=col_names, data=data.json()[1:])
            temp.append(data) #append the current instance to the last
        temp = pd.concat(temp)
        thing = temp
    data_appended.append(temp)
new_df = pd.concat(data_appended, axis = 1) 
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS', 'GeoFIPS2']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)


    
print('Your API call is complete')

Your API call is complete


In [152]:
new_df.head(2)

,NAME,GEO_ID,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaska

In [153]:
balances = new_df

In [154]:
balances['GEO_ID'] = '1'

In [155]:
balances.head()

,NAME,GEO_ID,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaska

In [156]:
msas = ['34980', '17300']
#this works msa
data_appended = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list = deque(col_list)
        col_list.appendleft(head2)  
        col_list.appendleft(head1)
        temp = []
        for msa in msas:
            predicates= {}
            get_vars= var_list
            predicates["get"]= ",". join(get_vars)
            predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
            data = requests.get(url_str, params= predicates)
            col_names = col_list
            data = pd.DataFrame(columns=col_names, data=data.json()[1:])
            temp.append(data) #append the current instance to the last
        temp = pd.concat(temp)
        thing = temp
    data_appended.append(temp)
new_df = pd.concat(data_appended, axis = 1) 
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)


    
print('Your API call is complete')

Your API call is complete


In [157]:
msas = new_df

In [158]:
dfs = [part_one, balances, new_df]

In [159]:
data = pd.concat(dfs)

In [160]:
data = data.set_index(['NAME', 'GEO_ID']).transpose()
data.head()

NAME,"Meigs County, Tennessee","Montgomery County, Tennessee","Moore County, Tennessee","Obion County, Tennessee","Overton County, Tennessee","Pickett County, Tennessee","Polk County, Tennessee","Rhea County, Tennessee","Roane County, Tennessee","Rutherford County, Tennessee","Scott County, Tennessee","Sequatchie County, Tennessee","Shelby County, Tennessee","Smith County, Tennessee","Sullivan County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Unicoi County, Tennessee","Van Buren County, Tennessee","Warren County, Tennessee","Wayne County, Tennessee","Weakley County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Bedford County, Tennessee","Blount County, Tennessee","Cannon County, Tennessee","Cheatham County, Tennessee","Clay County, Tennessee","Crockett County, Tennessee","Decatur County, Tennessee","Jackson County, Tennessee","Knox County, Tennessee","Lawrence County, Tennessee","Loudon County, Tennessee","Macon County, Tennessee","Marshall County, Tennessee","Monroe County, Tennessee","Morgan County, Tennessee","Perry County, Tennessee","Putnam County, Tennessee","Robertson County, Tennessee","Sevier County, Tennessee","Stewart County, Tennessee","Tipton County, Tennessee","Union County, Tennessee","Washington County, Tennessee","White County, Tennessee","Anderson County, Tennessee","Benton County, Tennessee","Bledsoe County, Tennessee","Bradley County, Tennessee","Campbell County, Tennessee","Carroll County, Tennessee","Carter County, Tennessee","Chester County, Tennessee","Claiborne County, Tennessee","Cocke County, Tennessee","Coffee County, Tennessee","Cumberland County, Tennessee","Davidson County, Tennessee","DeKalb County, Tennessee","Dickson County, Tennessee","Dyer County, Tennessee","Fayette County, Tennessee","Fentress County, Tennessee","Franklin County, Tennessee","Gibson County, Tennessee","Giles County, Tennessee","Grainger County, Tennessee","Greene County, Tennessee","Grundy County, Tennessee","Hamblen County, Tennessee","Hamilton County, Tennessee","Hancock County, Tennessee","Hardeman County, Tennessee","Hardin County, Tennessee","Hawkins County, Tennessee","Haywood County, Tennessee","Henderson County, Tennessee","Henry County, Tennessee","Hickman County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Jefferson County, Tennessee","Johnson County, Tennessee","Lake County, Tennessee","Lauderdale County, Tennessee","Lewis County, Tennessee","Lincoln County, Tennessee","McMinn County, Tennessee","McNairy County, Tennessee","Madison County, Tennessee","Marion County, Tennessee","Maury County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Orlinda city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smy

In [161]:
numcols = list(data.columns)
numcols
data[numcols] = data[numcols].astype(float)

In [162]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
#Cheatham
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
#Dickson
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
#Houston
HoustInc = [data[('Erin city, Tennessee', '1600000US4724320')],data[('Houston County (part), Tennessee Ridge town, Tennessee', '1')]]
data[('Houston Incorporated', 'None')] = sum(HoustInc)
data[('Houston Unincorporated', 'None')] = data[('Houston County, Tennessee', '0500000US47083')] - data[('Houston Incorporated', 'None')]
#Humphreys
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
#Maury
MauryInc = [data[('Mount Pleasant city, Tennessee', '1600000US4751080')],data[('Columbia city, Tennessee', '1600000US4716540')],
            data[('Maury County (part), Spring Hill city, Tennessee', '1')]]
data[('Maury Incorporated', 'None')] = sum(MauryInc)
data[('Maury Unincorporated', 'None')] = data[('Maury County, Tennessee', '0500000US47119')] - data[('Maury Incorporated', 'None')]
#Montgomery
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]
#Robertson
RobInc = [data[('Adams city, Tennessee', '1600000US4700200')],data[('Greenbrier town, Tennessee', '1600000US4730960')],
          data[('Cedar Hill city, Tennessee', '1600000US4711980')],data[('Coopertown town, Tennessee', '1600000US4716980')], 
         data[('Cross Plains city, Tennessee', '1600000US4718420')],data[('Springfield city, Tennessee', '1600000US4770500')], 
          data[('Orlinda city, Tennessee', '1600000US4756020')],
         data[('Robertson County (part), Portland city, Tennessee', '1')],data[('Robertson County (part), Millersville city, Tennessee', '1')], 
         data[('Robertson County (part), White House city, Tennessee', '1')], data[('Robertson County (part), Ridgetop city, Tennessee', '1')]]
data[('Robertson Incorporated', 'None')] = sum(RobInc)
data[('Robertson Unincorporated', 'None')] = data[('Robertson County, Tennessee', '0500000US47147')] - data[('Robertson Incorporated', 'None')]
#Rutherford
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
#Stewart
data[('Stewart County (part), Tennessee Ridge town, Tennessee', '1')] = data[('Tennessee Ridge town, Tennessee', '1600000US4773460')] - data[('Houston County (part), Tennessee Ridge town, Tennessee', '1')]
StewInc = [data[('Cumberland City town, Tennessee', '1600000US4718820')],data[('Dover city, Tennessee', '1600000US4721400')],
            data[('Stewart County (part), Tennessee Ridge town, Tennessee', '1')]]
data[('Stewart Incorporated', 'None')] = sum(StewInc)
data[('Stewart Unincorporated', 'None')] = data[('Stewart County, Tennessee', '0500000US47161')] - data[('Stewart Incorporated', 'None')]
#Sumner
SumInc = [data[('Hendersonville city, Tennessee', '1600000US4733280')],data[('Mitchellville city, Tennessee', '1600000US4749460')],
          data[('Westmoreland town, Tennessee', '1600000US4779420')],data[('Gallatin city, Tennessee', '1600000US4728540')], 
         data[('Sumner County (part), Portland city, Tennessee', '1')],data[('Sumner County (part), Millersville city, Tennessee', '1')], 
         data[('Sumner County (part), White House city, Tennessee', '1')],data[('Sumner County (part), Goodlettsville city, Tennessee', '1')]]
data[('Sumner Incorporated', 'None')] = sum(SumInc)
data[('Sumner Unincorporated', 'None')] = data[('Sumner County, Tennessee', '0500000US47165')] - data[('Sumner Incorporated', 'None')]
#Williamson
WilliInc = [data[('Nolensville town, Tennessee', '1600000US4753460')],data[('Brentwood city, Tennessee', '1600000US4708280')],
          data[('Fairview city, Tennessee', '1600000US4725440')],data[('Franklin city, Tennessee', '1600000US4727740')], 
         data[("Thompson's Station town, Tennessee", '1600000US4773900')],data[('Williamson County (part), Spring Hill city, Tennessee', '1')]]
data[('Williamson Incorporated', 'None')] = sum(WilliInc)
data[('Williamson Unincorporated', 'None')] = data[('Williamson County, Tennessee', '0500000US47187')] - data[('Williamson Incorporated', 'None')]
#Wilson
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]

In [163]:
data = data.transpose().reset_index()
data.tail()

,NAME,GEO_ID,pop,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_tworaces,raceeth_tworaces_white_blackafricanamerican,raceeth_tworaces_white_americanindianalaskanative,raceeth_tworaces_white_asian,raceeth_tworaces_white_nativehawaiianpacificislander,raceeth_tworaces_white_someotherrace,raceeth_tworaces_blackafricanamerican_americanindianalaskanative,raceeth_tworaces_blackafricanamerican_asian,raceeth_tworaces_blackafricanamerican_nativehawaiianpacificislander,raceeth_tworaces_blackafricanamerican_someotherrace,raceeth_tworaces_americanindianalaskanative_asian,raceeth_tworaces_americanindianalaskanative_nativehawaiianpacificislander,raceeth_tworaces_americanindianalaskanative_someotherrace,raceeth_tworaces_asiannativehawaiianpacificislander,raceeth_tworaces_asiansomeotherrace,raceeth_tworaces_nativehawaiianotherpacificislandersomeotherrace,raceeth_threeraces,raceeth_threeraces_white_blackafricanamerican_americanindianalaskanative,raceeth_threeraces_white_blackafricanamerican_asian,raceeth_threeraces_white_blackafricanamerican_nativehawaiianpacificislander,raceeth_threeraces_white_blackafricanamerican_someotherrace,raceeth_threeraces_white_americanindianalaskanative_asian,raceeth_threeraces_white_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_white_americanindianalaskanative_someotherrace,raceeth_threeraces_white_asian_nativehawaiianpacificislander,raceeth_threeraces_white_asian_someotherrace,raceeth_threeraces_white_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_asian,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_threeraces_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_threeraces_blackafricanamerican_asian_someotherrace,raceeth_threeraces_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_threeraces_americanindianalaskanative_asian_someotherrace,raceeth_threeraces_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_threeraces_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_asian,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_americanindianalaskanative_someotherrace,raceeth_fourraces_white_blackafricanamerican_asian_nativehawaiianpacificislander,raceeth_fourraces_white_blackafricanamerican_asian_someotherrace,raceeth_fourraces_white_blackafricanamerican_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_white_americanindianalaskanative_asian_someotherrace,raceeth_fourraces_white_americanindianalaskanative_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_white_asian_nativehawaiianpacificislander_someotherrace,raceeth_fourraces_blackafricanamerican_americanindianalaskanative_asian_nativehawaiianpacificislander,raceeth_fourraces_blackafricanamerican_americanindianalaska

In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 89 columns):
 #   Column                                                                                                                    Non-Null Count  Dtype  
---  ------                                                                                                                    --------------  -----  
 0   NAME                                                                                                                      199 non-null    object 
 1   GEO_ID                                                                                                                    199 non-null    object 
 2   pop                                                                                                                       199 non-null    float64
 3   raceeth_whitealone_nothispanicorlatino                                                                                    199 non-null 

In [165]:
data.to_feather('../../Raw Data/PL2020')